In [1]:
import numpy as np
import pandas as pd
import warnings
#import itertools
import time
import datetime
import math
from time import strftime
import numpy.random as random
from numpy import hstack, vstack
import time
import matplotlib.pyplot as plt
%matplotlib inline
file_dir = 'Loan_Forecasting_Qualification/'
today = strftime("%Y%m%d")

In [2]:
# from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import VarianceThreshold 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.externals import joblib
from sklearn import cross_validation, metrics
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

D:\programFiles\anaconda\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\programFiles\anaconda\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


#### 数据预处理

文件名1|字段名|字段描述|文件名2|字段名|字段描述
------|------|--------
t_user|uid|用户ID|t_order|uid|用户ID
    |age|年龄段||buy_time|购买时间
    |sex|||price|价格
    |active_date|用户激活日期||cate_id|品类id
   |limit|初始额度||discount|优惠金额
   ||||qty|购买数量
t_click|click_time|点击时间|t_loan|loan_time|借款时间
|pid|点击页面||loan_amount|借款金额
|param|页面参数||plannum|分期期数
t_loan_sum|month|统计月份|t_loan_sum|loan_sum|借款金额


In [3]:
#读取数据
t_user = pd.read_csv(file_dir+'t_user.csv')  #用户信息表，用户量90993
t_order = pd.read_csv(file_dir+'t_order.csv')  #订单信息表，用户量89244
t_click = pd.read_csv(file_dir+'t_click.csv')  #点击信息表，用户量78439
t_loan = pd.read_csv(file_dir+'t_loan.csv')   #借款信息表，用户量36697,其中pid=3,6偏好不明显，pid=8的偏好较明显,明显不明显主要以是否高于平均占比为标准
# #其中param=5,21,38 偏好不明显，param=26,28,36的偏好较明显
t_loan_sum = pd.read_csv(file_dir+'t_loan_sum.csv')  #月借款总额表，用户量19520

In [4]:
def rename_columns(pre,orig_cols):
    #orig_cols = a.columns.levels
    if len(orig_cols[1])==4:  #传入的为分箱类pivot的cols
        cols_list = ['_lv1','_lv2','_lv3','_lv4']
        cols_list = [pre + col for col in cols_list]
    if len(orig_cols[1])==3:
        cols_list = ['_mon1','_mon2','_mon3']
        cols_list = [pre + col for col in cols_list] 
    columns = []
    for i in list(cols_list):
        for j in list(orig_cols[0]):
            columns.append(str(i)+'_'+ str(j))
    return columns

def agg_fun(data,columns,agg_list):
        
    data = data[columns]
    agg_data = data.groupby('uid').agg(agg_list)
    new_columns = []
    for i in agg_data.columns.levels[1]:
        for j in agg_data.columns.levels[0]:
            new_columns.append(str(j)+ '_'+ str(i))
    agg_data.columns = new_columns
    agg_data = agg_data.reset_index()

    return agg_data

#### t_order表的处理

In [ ]:
#提前处理
t_order = pd.read_csv(file_dir+'t_order.csv')  #订单信息表，用户量89244
t_order['price'] = t_order['price'].apply(lambda x: math.pow(5,x)-1)*t_order['qty']
t_order['buy_price'] = round(t_order['price']*(10-t_order['discount'])*0.1,1)
t_order['privilege'] = round((t_order['price'] - t_order['buy_price']),2)
t_order['mon'] = t_order['buy_time'].apply(lambda x: x[5:7])

cate_price_np = t_order.groupby(['cate_id']).median()['price']
cate_price = pd.DataFrame(t_order.groupby(['cate_id']).median()['price']).reset_index()
cate_price['cate_high'] = 0
bins = [int(cate_price_np.min())-1,int(np.percentile(cate_price_np,25)),int(np.percentile(cate_price_np,50)),int(np.percentile(cate_price_np,75)),int(cate_price_np.max())+1]
cats = pd.cut(cate_price.price, bins)
a = pd.DataFrame([list(cate_price.price),list(cats)]).T
cate_price['cate_bins'] = a[1]
cate_price = cate_price.rename(columns={'price':'cate_price'})
t_order = pd.merge(t_order,cate_price,on='cate_id',how='left')
t_order['cate_bins'] = t_order.cate_bins.astype('str')
t_order['cate_bins'] = t_order['cate_bins'].replace({'(-1, 49]':'cate_lv1','(49, 86]':'cate_lv2',
                                                    '(86, 192]':'cate_lv3','(192, 1000000000]':'cate_lv4'})

def buy_month(data,time_fea,start_day='2016-08-01',end_day='2016-11-30',group_fea = 'uid'):
    
    """
    time_fea :根据时间范围筛选数据的特征
    start_day:time_fea筛选时间的起点,{t_order:'2016-08-03'}
    end_day:end_day筛选时间的终点
    group_fea：用于分组计算的特征，通常为uid
    
    """
    data0 = data
    data = data[(data[time_fea]<=end_day)&(data[time_fea]>=start_day)]

    #通过时间获取变量，[agg_data_a]
#     columns_obj = ['uid'] + list(pd.DataFrame(data.dtypes)[pd.DataFrame(data.dtypes)[0]=='object'].index) #获取数据中object(时间类)类的列
    columns_obj = ['uid','buy_time']
    agg_data_a = agg_fun(data,columns_obj,['max','min']) 
    #最后一次购买距离现在的时长，并采取时长分箱
    agg_data_a['buy_time_diff'] = (pd.to_datetime(end_day)- pd.to_datetime(agg_data_a['buy_time_max']))/np.timedelta64(1,'D')
    bins = [agg_data_a.buy_time_diff.min()-1,agg_data_a.buy_time_diff.min()+3,agg_data_a.buy_time_diff.min()+7,agg_data_a.buy_time_diff.min()+14,agg_data_a.buy_time_diff.max()]  
    cats = pd.cut(agg_data_a.buy_time_diff, bins)  
    a = pd.DataFrame([list(agg_data_a.buy_time_diff),list(cats)]).T
    agg_data_a['buy_diff_bins'] = a[1]
    
    agg_data_bins = pd.merge(data,agg_data_a.loc[:,['uid','buy_diff_bins']],on='uid')
    
    bins_price = pd.pivot_table(agg_data_bins,index='uid',columns='buy_diff_bins',values='price',aggfunc={'max','mean','min','count','sum'})
    bins_price.columns = rename_columns('diff_price',bins_price.columns.levels)
    bins_price = bins_price.reset_index()
    
    bins_privg = pd.pivot_table(agg_data_bins,index='uid',columns='buy_diff_bins',values='privilege',aggfunc={'max','mean','min','count','sum'})
    bins_privg.columns = rename_columns('diff_privg',bins_privg.columns.levels)
    bins_privg = bins_privg.reset_index()
    
    mon_price = pd.pivot_table(agg_data_bins,index='uid',columns='mon',values='price',aggfunc={'max','mean','min','count','sum'})
    mon_price.columns = rename_columns('mon_price',mon_price.columns.levels)
    mon_price = mon_price.reset_index()
    
    mon_privg = pd.pivot_table(agg_data_bins,index='uid',columns='mon',values='privilege',aggfunc={'max','mean','min','count','sum'})
    mon_privg.columns = rename_columns('mon_privg',mon_privg.columns.levels)
    mon_privg = mon_privg.reset_index()
    
    cate_price = pd.pivot_table(agg_data_bins,index='uid',columns='cate_bins',values='price',aggfunc={'max','mean','min','count','sum'})
    cate_price.columns = rename_columns('cate_price',cate_price.columns.levels)
    cate_price = cate_price.reset_index()
    
#     columns = list(pd.DataFrame(data.dtypes)[pd.DataFrame(data.dtypes)[0]!='object'].index)  #剔除数据中object（时间类）的列
    columns = ['uid','price','qty','discount','privilege']
    agg_data_b = agg_fun(data,columns,['max','min','count','sum'])
    
    max_price = agg_data_b[['uid','price_max']].rename(columns= {'price_max':'price'})
    max_price_time = pd.merge(data0[['uid','buy_time','price']],max_price[['uid','price']],on=['uid','price'])
    agg_data_b['max_buy_time'] = max_price_time['buy_time']
    agg_data_b['is_high_consume'] = 0
    agg_data_b['is_high_consume'] = agg_data_b[agg_data_b.price_sum>=int(agg_data_b.price_sum.describe()[6:7])]['is_high_consume'].replace(0,1)
    agg_data_b['is_high_consume'] = agg_data_b['is_high_consume'].fillna(0)
    
    agg_data =  pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(agg_data_a,bins_price,on='uid',how='left'),bins_privg,on='uid',
             how='left'),mon_price,on='uid',how='left'),mon_privg,on='uid',how='left'),cate_price,on='uid',how='left'),agg_data_b,on='uid',how='left')
    
    return agg_data

#生成t_order表的特征
agg_data_10 = buy_month(t_order,'buy_time',end_day='2016-10-31')
agg_data_11 = buy_month(t_order,'buy_time',start_day='2016-09-01')
agg_data_10.to_hdf("{}/t_order_10.h5".format(file_dir), key = "t_order_10", complib = "zlib", complevel = 1, mode = "w")
agg_data_11.to_hdf("{}/t_order_11.h5".format(file_dir), key = "t_order_11", complib = "zlib", complevel = 1, mode = "w")

#### t_loan表处理

In [ ]:
t_loan = pd.read_csv(file_dir+'t_loan.csv') 
t_loan['loan_day'] = t_loan['loan_time'].map(lambda x:x[:10]) 
t_loan['loan_amount'] = t_loan['loan_amount'].apply(lambda x: math.pow(5,x)-1)
def loan_time(data,time_fea,start_day='2016-08-01',end_day='2016-11-30',group_fea = 'uid'):
    
    def agg_fun(data,columns,agg_list):
        
        data = data[columns]
        agg_data = data.groupby(group_fea).agg(agg_list)
        agg_data.columns = rename_columns(agg_data.columns.levels)
        agg_data = agg_data.reset_index()
        
        return agg_data
    
    data = data[(data[time_fea]<=end_day)&(data[time_fea]>=start_day)] 
    data['loan_mon'] = data['loan_time'].map(lambda x:x[5:7])
    data['loan_day'] = data['loan_time'].map(lambda x:x[:10])
    data['loan_v'] = data['loan_time'].apply(lambda x: time.mktime(time.strptime(x,'%Y-%m-%d %H:%M:%S')))
    #找到用户点击的总时间的差异
    t_loan_time = pd.DataFrame(data.groupby(['uid']).agg(['max','min'])['loan_v']).reset_index()
    t_loan_time['loan_hour_diff'] = (t_loan_time['max'] - t_loan_time['min'])/3600
    #t_click_time['day_diff'] = (t_click_time['max'] - t_click_time['min'])/3600/24
    t_loan_time = t_loan_time.drop(['max','min'],axis=1)

    #通过时间获取变量，[agg_data_a]
    columns_obj = ['uid','loan_day']  #获取数据中object(时间类)类的列
    agg_data_a = agg_fun(data,columns_obj,['max','min']) 
    #最后一次购买距离现在的时长，并采取时长分箱
    agg_data_a['loan_day_diff'] = (pd.to_datetime(end_day)- pd.to_datetime(agg_data_a['loan_day_max']))/np.timedelta64(1,'D')
    bins = [agg_data_a.loan_day_diff.min()-1,agg_data_a.loan_day_diff.min()+3,agg_data_a.loan_day_diff.min()+7,agg_data_a.loan_day_diff.max()]  
    cats = pd.cut(agg_data_a.loan_day_diff, bins)  
    a = pd.DataFrame([list(agg_data_a.loan_day_diff),list(cats)]).T
    agg_data_a['loan_diff_bins'] = a[1]
    
    agg_data_bins = pd.merge(data,agg_data_a.loc[:,['uid','loan_diff_bins']],on='uid')
    
    
    #按时间分箱汇总数据 [agg_data_b]
    bins_loan_cnt = get_pivot_table(agg_data_bins,'loan_diff_bins','plannum',aggf='count',cols_pre=['click_loan_cnt_3','click_loan_cnt_7','click_loan_cnt_max'])
    bins_loan_avg = get_pivot_table(agg_data_bins,'loan_diff_bins','loan_amount',aggf='mean',cols_pre=['click_loan_avg_3','click_loan_avg_7','click_loan_avg_max'])
    bins_loan_max = get_pivot_table(agg_data_bins,'loan_diff_bins','loan_amount',aggf='max',cols_pre=['click_loan_max_3','click_loan_max_7','click_loan_max_max'])
    
    mon_loan_cnt = get_pivot_table(agg_data_bins,'loan_mon','loan_amount',aggf='count',cols_pre=['click_loan_near_cnt_1','click_loan_near_cnt_2','click_loan_near_cnt_3'])
    mon_loan_avg = get_pivot_table(agg_data_bins,'loan_mon','loan_amount',aggf='mean',cols_pre=['click_loan_near_avg_1','click_loan_near_avg_2','click_loan_near_avg_3'])
    mon_loan_max = get_pivot_table(agg_data_bins,'loan_mon','loan_amount',aggf='max',cols_pre=['click_loan_near_max_1','click_loan_near_max_2','click_loan_near_max_3'])
    
    bin_data = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(bins_loan_cnt,bins_loan_avg,on='uid',how='left'),bins_loan_max,on='uid',how='left'),mon_loan_cnt,on='uid',how='left'),
                        mon_loan_avg,on='uid',how='left'),mon_loan_max,on='uid',how='left')
    
    bins_num1 = agg_data_bins[agg_data_bins.plannum==1]
    bins_num1_cnt = get_pivot_table(bins_num1,'loan_diff_bins','plannum',aggf='count',cols_pre=['loan_num1_cnt_3','loan_num1_cnt_7','loan_num1_cnt_max'])
    bins_num3 = agg_data_bins[agg_data_bins.plannum==3]
    bins_num3_cnt = get_pivot_table(bins_num3,'loan_diff_bins','plannum',aggf='count',cols_pre=['loan_num3_cnt_3','loan_num3_cnt_7','loan_num3_cnt_max'])
    bins_num6 = agg_data_bins[agg_data_bins.plannum==6]
    bins_num6_cnt = get_pivot_table(bins_num6,'loan_diff_bins','plannum',aggf='count',cols_pre=['loan_num6_cnt_3','loan_num6_cnt_7','loan_num6_cnt_max'])
    bins_num12 = agg_data_bins[agg_data_bins.plannum==12]
    bins_num12_cnt = get_pivot_table(bins_num12,'loan_diff_bins','plannum',aggf='count',cols_pre=['loan_num12_cnt_3','loan_num12_cnt_7','loan_num12_cnt_max'])
    
    agg_data_b = pd.merge(pd.merge(pd.merge(pd.merge(bin_data,bins_num1_cnt,on='uid'),bins_num3_cnt,on='uid',how='left'),bins_num6_cnt,on='uid',how='left'),bins_num12_cnt,on='uid',how='left')
    
    #统计目前为止分期情况
    loan_user_plan = pd.pivot_table(data,index='uid',columns='plannum',values='loan_amount',aggfunc='count')
    loan_user_plan.columns = [str(i) + '_num' for i in list(loan_user_plan.columns)]
    loan_user_plan = loan_user_plan.reset_index()
    
    #统计预测前一个月分期情况
   


    
    agg_data = pd.merge(pd.merge(pd.merge(t_loan_time,agg_data_a,on='uid',how='left'),agg_data_b,on='uid',how='left'),loan_user_plan,on='uid',how='left')
    
    return agg_data

# t_loan_agg_10 = loan_time(t_loan,'loan_day',start_day='2016-08-01',end_day='2016-10-31',group_fea = 'uid')
# t_loan_agg_10.to_hdf("{}/t_loan_agg_10.h5".format(file_dir), key = "t_loan_agg_10", complib = "zlib", complevel = 1, mode = "w")
t_loan_agg_11 = loan_time(t_loan,'loan_day',start_day='2016-09-01',end_day='2016-11-30',group_fea = 'uid')
t_loan_agg_11.to_hdf("{}/t_loan_agg_11.h5".format(file_dir), key = "t_loan_agg_11", complib = "zlib", complevel = 1, mode = "w")

#### t_click表处理

In [ ]:
import time
t_click = pd.read_csv(file_dir+'t_click.csv')  #点击信息表，用户量78439
t_click['click_day'] = t_click['click_time'].map(lambda x:x[:10])

def click_time(data,time_fea,start_day='2016-08-01',end_day='2016-11-30',group_fea = 'uid'):
    
    def agg_fun(data,columns,agg_list):
        
        data = data[columns]
        agg_data = data.groupby(group_fea).agg(agg_list)
        agg_data.columns = rename_columns(agg_data.columns.levels)
        agg_data = agg_data.reset_index()
        
        return agg_data
    
    data = data[(data[time_fea]<=end_day)&(data[time_fea]>=start_day)]
    data['click_mon'] = data['click_time'].map(lambda x:x[5:7])
    data['click_time_v'] = data['click_time'].apply(lambda x: time.mktime(time.strptime(x,'%Y-%m-%d %H:%M:%S')))
   
    #找到用户点击的总时间的差异[t_click_time]
    t_click_time = pd.DataFrame(data.groupby(['uid']).agg(['max','min'])['click_time_v']).reset_index()
    t_click_time['click_hour_diff'] = (t_click_time['max'] - t_click_time['min'])/3600
    t_click_time = t_click_time.drop(['max','min'],axis=1)
    
    
    #通过时间获取变量，[agg_data_a]
    columns_obj = ['uid','click_day']  #获取数据中object(时间类)类的列
    agg_data_a = agg_fun(data,columns_obj,['max','min']) 
    #最后一次购买距离现在的时长，并采取时长分箱
    agg_data_a['click_day_diff'] = (pd.to_datetime(end_day)- pd.to_datetime(agg_data_a['click_day_max']))/np.timedelta64(1,'D')
    bins = [agg_data_a.click_day_diff.min()-1,agg_data_a.click_day_diff.min()+3,agg_data_a.click_day_diff.min()+7,agg_data_a.click_day_diff.max()]  
    cats = pd.cut(agg_data_a.click_day_diff, bins)  
    a = pd.DataFrame([list(agg_data_a.click_day_diff),list(cats)]).T
    agg_data_a['click_diff_bins'] = a[1]
    
    agg_data_bins = pd.merge(data,agg_data_a.loc[:,['uid','click_diff_bins']],on='uid')
    
    
    #按时间分箱汇总数据 [agg_data_b]
    bins_pid_cnt = get_pivot_table(agg_data_bins,'click_diff_bins','pid',aggf='count',cols_pre=['click_pid_cnt_3','click_pid_cnt_7','click_pid_cnt_max'])
    mon_pid_cnt = get_pivot_table(agg_data_bins,'click_mon','pid',aggf='count',cols_pre=['click_pid_near_cnt_1','click_pid_near_cnt_7','click_pid_near_cnt_max'])
    bins_pid6 = agg_data_bins[agg_data_bins.pid==6]
    bins_pid6_cnt = get_pivot_table(bins_pid6,'click_diff_bins','pid',aggf='count',cols_pre=['click_pid6_cnt_3','click_pid6_cnt_7','click_pid6_cnt_max'])
    bins_pid8 = agg_data_bins[agg_data_bins.pid==8]
    bins_pid8_cnt = get_pivot_table(bins_pid8,'click_diff_bins','pid',aggf='count',cols_pre=['click_pid8_cnt_3','click_pid8_cnt_7','click_pid8_cnt_max'])
    bins_pid10 = agg_data_bins[agg_data_bins.pid==10]
    bins_pid10_cnt = get_pivot_table(bins_pid10,'click_diff_bins','pid',aggf='count',cols_pre=['click_pid10_cnt_3','click_pid10_cnt_7','click_pid10_cnt_max'])
    
    agg_data_b = pd.merge(pd.merge(pd.merge(pd.merge(bins_pid_cnt,mon_pid_cnt,on='uid'),bins_pid6_cnt,on='uid',how='left'),bins_pid8_cnt,on='uid',how='left'),bins_pid10_cnt,on='uid',how='left')
    
    
    # 重点 pid特征提取[agg_data_pid]
    agg_data_pid = data[data['pid'].isin([6,8,10])]
    agg_data_pid = pd.pivot_table(agg_data_pid,index='uid',columns='pid',values='param',aggfunc='count')
    agg_data_pid.columns = ['pid_'+ str(i)  for i in list(agg_data_pid.columns)]
    agg_data_pid = agg_data_pid.reset_index()
    agg_data_pid = agg_data_pid.fillna(0)
    
    #重点param特征提取[agg_data_param]
    agg_data_param = data[data['param'].isin([16,30,1])]
    agg_data_param = pd.pivot_table(agg_data_param,index='uid',columns='param',values='pid',aggfunc='count')
    agg_data_param.columns = ['param_'+ str(i)  for i in list(agg_data_param.columns)]
    agg_data_param = agg_data_param.reset_index()
    agg_data_param = agg_data_param.fillna(0)
    
    agg_data = pd.merge(pd.merge(pd.merge(pd.merge(t_click_time,agg_data_a,on='uid',how='left'),agg_data_b,on='uid',how='left'),agg_data_pid,on='uid',how='left'),agg_data_param,on='uid',how='left')
    
    
    return agg_data


In [ ]:
t_user = pd.read_csv(file_dir+'t_user.csv')  #用户信息表，用户量90993
t_loan_sum = pd.read_csv(file_dir+'t_loan_sum.csv')
t_click_10 = pd.read_hdf("{}/t_click_agg_10.h5".format(file_dir), key = "t_click_agg_10")
t_order_10 = pd.read_hdf("{}/t_order_agg10.h5".format(file_dir), key = "t_order_agg10")
t_loan_10 = pd.read_hdf("{}/t_loan_agg_10.h5".format(file_dir), key = "t_loan_agg_10")

#### 数据合并

In [ ]:
#sample_demand
sample_demand = list(set(t_click_10.uid))
sample_demand = pd.DataFrame(sample_demand).rename(columns={0:'uid'})
sample_demand['is_demand'] = 0
sample_demand['is_demand'] = sample_demand[sample_demand['uid'].isin(set(t_loan_sum.uid))]['is_demand'].replace(0,1)
sample_demand['is_demand'] = sample_demand['is_demand'].fillna(0)
sample_demand.is_demand = sample_demand.is_demand.astype('int32')
sample_demand = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(sample_demand,t_user,on='uid',how='left'),t_order_10,on='uid',how='left'),
                                           t_click_10,on='uid',how='left'),t_loan_10,on='uid',how='left'),t_loan_sum.loc[:,['uid','loan_sum']],on='uid',how='left')
sample_demand['active_buy_diff'] = (pd.to_datetime(sample_demand.loc[:,'buy_time_min'])- pd.to_datetime(sample_demand.loc[:,'active_date']))/np.timedelta64(1,'D')
sample_demand.to_hdf("{}/sample_demand_v2.h5".format(file_dir), key = "sample_demand_v2", complib = "zlib", complevel = 1, mode = "w")

In [ ]:
t_click_11 = pd.read_hdf("{}/t_click_agg_11.h5".format(file_dir), key = "t_click_agg_11")
t_order_11 = pd.read_hdf("{}/t_order_agg_11.h5".format(file_dir), key = "t_order_agg_11")
t_loan_11 = pd.read_hdf("{}/t_loan_agg_11.h5".format(file_dir), key = "t_loan_agg_11")


pred_demand = pd.merge(pd.merge(pd.merge(t_user,t_order_11,on='uid',how='left'),t_click_11,on='uid',how='left'),t_loan_11,on='uid',how='left')
pred_demand['active_buy_diff'] = (pd.to_datetime(pred_demand.loc[:,'buy_time_min'])- pd.to_datetime(pred_demand.loc[:,'active_date']))/np.timedelta64(1,'D')
# pred_demand.to_csv(file_dir+'pred_demand.csv',index = False)
#拼特征9-11月份，预测样本特征2
# loan_time_11 = loan_time(t_loan,'loan_time',start_day='2016-09-01')
# pred_loan = pd.merge(pred_demand,loan_time_11,on='uid',how='left')
# pred_loan.to_csv(file_dir+'pred_loan.csv',index=False)
pred_demand.to_hdf("{}/pred_demand_v2.h5".format(file_dir), key = "pred_demand_v2", complib = "zlib", complevel = 1, mode = "w")

#### build model

In [3]:
def train_cla_model(sample_loan):
    target = ['uid','is_demand','loan_sum']
    predictors = [x for x in sample_loan.columns if x not in target]
    X = sample_loan.loc[:,predictors]
    y = sample_loan.loc[:,['is_demand']]
    X_train,X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size=0.4, random_state=1)
    xgb1 = XGBClassifier(learning_rate =0.01,n_estimators=500, max_depth=5,\
                    min_child_weight=5,gamma=0,subsample=0.8,colsample_bytree=0.8,\
                    objective= 'binary:logistic', nthread=4,scale_pos_weight=1,seed=27)
    xgb1 = xgb1.fit(X_train,y_train)
    y_pred = xgb1.predict_proba(X_test)[:,1]
    print('the auc of the classifier model is:',roc_auc_score(y_test.values,y_pred))
#     score = cross_validation.cross_val_score(xgb1,X,y,cv=5,scoring='roc_auc')
    return xgb1

def read_pred_demand_data(xgb,pred_loan):
    
    pred_demand_result = pred_loan.loc[:,['uid']]
    predictors = [x for x in pred_loan.columns if x not in ['uid']]
    pred_loan = pred_loan.loc[:,predictors]
    demand_prob = xgb.predict_proba(pred_loan)[:,1]
    is_demand = xgb.predict(pred_loan)
    is_demand_11 = pd.concat([pred_demand_result,pd.DataFrame(list(is_demand))],axis=1)
    is_demand_11.columns = ['uid','is_demand']
#     threshold = round(float(is_demand_11.describe().reset_index().loc[6:6,['demand_prob']].values),4)
#     is_demand_11_06 = is_demand_11[is_demand_11.demand_prob < threshold].replace(1,0)
#     is_demand_11_0 = is_demand_11[is_demand_11.demand_prob >= threshold]
#     is_demand = pd.concat([is_demand_11_06,is_demand_11_0],axis=0)
    
    return is_demand_11

def train_reg_model(sample_loan):
    target = ['uid','loan_sum']
    predictors = [x for x in sample_loan.columns if x not in target]
    X = sample_loan.loc[:,predictors]
    X = X[predictors].fillna(0)
#     y = sample_loan['loan_sum'].apply(lambda x: math.pow(5,x)-1)  #
    y = sample_loan.loc[:,'loan_sum']
    X_train,X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size=0.6, random_state=2)
    xgb_liner = xgb.XGBRegressor(learning_rate =0.01,n_estimators=500,max_depth=5,min_child_weight=7,gamma=0.1,subsample=0.6,
                             colsample_bytree=0.6,reg_alpha=1,objective= 'reg:linear',nthread=4,scale_pos_weight=1,seed=1)
    xgb_liner.fit(X_train,y_train)
    y_pred = xgb_liner.predict(X_test)
#     y_pred = pd.DataFrame(y_pred).loc[:,0].apply(lambda x: x if x>=0 else 0 )
    y_pred = pd.DataFrame(y_pred).loc[:,0].apply(lambda x: abs(x) )
    print('the result of regression is:',math.sqrt(mean_squared_error(y_test, y_pred)))
    
    feat_imp = pd.Series(xgb_liner.feature_importances_)
    feat_imp_result = pd.DataFrame([predictors,list(feat_imp)]).T
#     feat_imp = feat_imp.sort_values(ascending=False)
#     feat_imp.plot(kind='bar', title='Feature Importances')
#     plt.ylabel('Feature Importance Score')
    
    return xgb_liner,pd.concat([y_pred,y_test],axis=1),feat_imp_result

def get_result(xgb,pred_loan):
    target = ['uid','loan_sum']
    predictors = [x for x in pred_loan.columns if x not in target]
#     no_demand_users = list(pred_loan[pred_loan.is_demand==0].uid)
    pred_loan_result = pred_loan.loc[:,['uid']]
    pred_loan = pred_loan.loc[:,predictors]
    y_pred = xgb.predict(pred_loan)
    y_pred = pd.DataFrame(y_pred).loc[:,0].apply(lambda x: x if x>=0 else 0 ) #
    pred_loan_result['loan_sum'] = y_pred
#     pred_loan_result_1 = pred_loan_result[pred_loan_result['uid'].isin(no_demand_users)|(pred_loan_result.loan_sum<=0)]
#     pred_loan_result_1['loan_sum'] = 0 
#     pred_loan_result_2 = pred_loan_result[~(pred_loan_result.loc[:,'uid'].isin(no_demand_users)|(pred_loan_result.loan_sum<=0))]
    
#     result = pd.concat([pred_loan_result_1,pred_loan_result_2],axis=0)
    
    return pred_loan_result

In [4]:
sample_demand = pd.read_hdf("{}/sample_demand_v2_3.h5".format(file_dir), key = "sample_demand_v2_3")
pred_demand = pd.read_hdf("{}/pred_demand_v2_3.h5".format(file_dir), key = "pred_demand_v2_3")

In [46]:
#数据窗口预测试一下
# sample_demand = pd.read_hdf("{}/data_v3/agg_data/sample_demand_v3.h5".format(file_dir), key = "sample_demand_v3")
# pred_demand = pd.read_hdf("{}/data_v3/agg_data/pred_loan_v3.h5".format(file_dir), key = "pred_loan_v3")
# sample_demand = sample_demand.drop('is_demand',axis=1)

In [47]:
# predictors = list(pd.DataFrame(sample_demand.dtypes)[pd.DataFrame(sample_demand.dtypes)[0]!='object'].index)
# sample_demand = sample_demand.loc[:,predictors]

In [48]:
# predictors = [pre for pre in predictors if pre not in ['is_demand','loan_sum']]
# pred_demand = pred_demand.loc[:,predictors]

In [49]:
len(sample_demand.columns),len(pred_demand.columns)

(244, 243)

In [17]:
list(sample_demand.columns)

['12_num',
 '1_num',
 '3_num',
 '6_num',
 'buy_diff_bins',
 'buy_price_avg_14',
 'buy_price_avg_3',
 'buy_price_avg_7',
 'buy_price_avg_max',
 'buy_price_cnt_14',
 'buy_price_cnt_3',
 'buy_price_cnt_7',
 'buy_price_cnt_max',
 'buy_price_count',
 'buy_price_max',
 'buy_price_max_14',
 'buy_price_max_3',
 'buy_price_max_7',
 'buy_price_max_max',
 'buy_price_min',
 'buy_price_min_14',
 'buy_price_min_3',
 'buy_price_min_7',
 'buy_price_min_max',
 'buy_price_sum',
 'buy_privg_avg_14',
 'buy_privg_avg_3',
 'buy_privg_avg_7',
 'buy_privg_avg_max',
 'buy_privg_cnt_14',
 'buy_privg_cnt_3',
 'buy_privg_cnt_7',
 'buy_privg_cnt_max',
 'buy_privg_max_14',
 'buy_privg_max_3',
 'buy_privg_max_7',
 'buy_privg_max_max',
 'buy_privg_min_14',
 'buy_privg_min_3',
 'buy_privg_min_7',
 'buy_privg_min_max',
 'buy_time_diff',
 'buy_time_max',
 'buy_time_min',
 'cate_high_count',
 'cate_high_max',
 'cate_high_min',
 'cate_high_sum',
 'cate_id_count',
 'cate_id_max',
 'cate_id_min',
 'cate_id_sum',
 'cateid_1'

In [5]:
######################################################
#版本1的变量：
# predictors = ['uid','click_hour_diff','click_pid8_cnt_max','click_day_diff','loan_hour_diff',
#                 'loan_day_diff','bins_amt_mon1_count','bins_amt_mon1_max','bins_amt_mon1_mean','bins_amt_mon_2_max',
#                 'bins_amt_mon_2_sum','bins_amt_mon3_count','bins_amt_mon3_mean','bins_amt_mon3_min','bins_amt_mon3_sum',
#                 'mons_amt_mon1_count','mons_amt_mon1_max','mons_amt_mon1_mean','mons_amt_mon3_max','mons_amt_mon3_mean',
#                 'mons_amt_mon3_min','mons_amt_mon3_sum','bins_num1_mon1_count','bins_num1_mon1_max','bins_num1_mon1_mean',
#                 'bins_num1_mon1_sum','bins_num1_mon_2_count','bins_num1_mon_2_sum','bins_num1_mon3_mean','bins_num1_mon3_min',
#                 'bins_num1_mon3_sum','bins_num3_mon1_count','bins_num3_mon1_max','bins_num3_mon1_mean','bins_num3_mon3_mean',
#                 'bins_num3_mon3_sum','bins_num6_mon1_count','bins_num6_mon1_max','bins_num6_mon1_mean','bins_num6_mon_2_max',
#                 'bins_num6_mon_2_mean','bins_num6_mon_2_sum','bins_num6_mon3_count','bins_num12_mon1_count','bins_num12_mon1_max',
#                 'bins_num12_mon1_mean','bins_num12_mon1_min','bins_num12_mon1_sum','bins_num12_mon_2_max','bins_num12_mon_2_mean',
#                 'bins_num12_mon_2_min','bins_num12_mon_2_sum','bins_num12_mon3_count','bins_num12_mon3_max','bins_num12_mon3_mean',
#                 'bins_num12_mon3_min','bins_num12_mon3_sum','mon_num1_mon1_count','mon_num1_mon1_max','mon_num1_mon1_mean',
#                 'mon_num1_mon1_min','mon_num1_mon1_sum','mon_num1_mon_2_count','mon_num1_mon_2_min','mon_num1_mon3_mean',
#                 'mon_num1_mon3_min','mon_num1_mon3_sum','mon_num3_mon1_count','mon_num3_mon1_max','mon_num3_mon1_mean',
#                 'mon_num6_mon1_count','mon_num6_mon1_max','mon_num6_mon1_mean','mon_num6_mon_2_mean','mon_num6_mon3_count',
#                 'mon_num6_mon3_max','mon_num12_mon1_count','mon_num12_mon1_max','mon_num12_mon1_mean','mon_num12_mon_2_count',
#                 'mon_num12_mon_2_max','mon_num12_mon_2_mean','mon_num12_mon_2_min','mon_num12_mon_2_sum','mon_num12_mon3_count',
#                 'mon_num12_mon3_max','1_num','loan_sum','daydiff5','daydiff12','daydiff13','daydiff19','daydiff20','daydiff32',
#                 'daydiff33','daydiff34','daydiff40','daydiff41'
#             ]
######################################################

# 目前最优！版本2的变量：与loan_sum的相关性0.2以上的变量
predictors = ['uid','loan_hour_diff','loan_day_diff','bins_amt_mon1_count','bins_amt_mon1_max','bins_amt_mon1_mean','bins_amt_mon_2_count',
              'bins_amt_mon_2_min','bins_amt_mon3_max','bins_amt_mon3_mean','bins_amt_mon3_min','mons_amt_mon1_count','mons_amt_mon1_max',
              'mons_amt_mon1_mean','mons_amt_mon3_mean','mons_amt_mon3_min','mons_amt_mon3_sum','bins_num1_mon1_count','bins_num1_mon1_max',
              'bins_num1_mon1_mean','bins_num1_mon1_min','bins_num1_mon1_sum','bins_num1_mon_2_count','bins_num1_mon3_mean','bins_num1_mon3_min',
              'bins_num1_mon3_sum','bins_num12_mon_2_count','bins_num12_mon_2_min','bins_num12_mon3_max','bins_num12_mon3_sum','mon_num1_mon1_count',
              'mon_num1_mon1_max','mon_num1_mon1_mean','mon_num1_mon1_min','mon_num1_mon1_sum','mon_num1_mon_2_count','mon_num1_mon_2_max','mon_num1_mon_2_mean',
              'mon_num1_mon_2_min','mon_num1_mon_2_sum','mon_num1_mon3_count','mon_num1_mon3_max','mon_num1_mon3_mean','mon_num1_mon3_min','mon_num1_mon3_sum',
              'mon_num12_mon_2_count','mon_num12_mon_2_mean','mon_num12_mon_2_min','mon_num12_mon3_count','mon_num12_mon3_max','1_num','3_num','daydiff5','daydiff6',
              'daydiff12','daydiff20','daydiff45','daydiff53','daydiff54','loan_sum'
             ]

###################################################
# 版本三的变量：效果没版本1好
# drop_cols_1 = ['daydiff7','daydiff15','daydiff23','daydiff31','daydiff39','daydiff47','daydiff55','daydiff14','daydiff21',
#             'daydiff28','daydiff35','daydiff42','daydiff49','daydiff22','daydiff29','daydiff36','daydiff43','daydiff50',
#             'daydiff24','daydiff25','daydiff26','daydiff27','daydiff32','daydiff33','daydiff34','daydiff40','daydiff41',
#             'daydiff48','diff_price_lv1_count','diff_price_lv1_max','diff_price_lv1_mean','diff_price_lv1_min','diff_price_lv2_mean',
#             'diff_privg_lv1_max','diff_privg_lv1_mean','diff_privg_lv1_min','diff_privg_lv1_count']
# drop_cols_2 = drop_cols_1 + ['is_demand']
# sample_demand = sample_demand.drop(drop_cols_2,axis=1)
# pred_demand = pred_demand.drop(drop_cols_1,axis=1)
# predictors =  list(sample_demand.columns)
# sample_demand = sample_demand.loc[:,predictors]

##################################################

#版本4的变量：综合xgboost跑出的前70变量+版本1变量+版本2变量
# predictors = ['uid','pid_8', 'pid_6', 'pid_10', 'param_16', 'param_1', 'mons_amt_mon3_sum', 'mons_amt_mon3_min', 'mons_amt_mon3_mean', 'mons_amt_mon3_max',
#               'mons_amt_mon1_mean', 'mons_amt_mon1_max', 'mons_amt_mon1_count', 'mons_amt_mon_2_min', 'mon_privg_mon_2_mean', 'mon_privg_mon_2_max',
#               'mon_price_mon3_sum', 'mon_price_mon3_min', 'mon_price_mon3_max', 'mon_price_mon3_count', 'mon_price_mon1_sum', 'mon_price_mon_2_sum',
#               'mon_price_mon_2_min', 'mon_price_mon_2_mean', 'mon_price_mon_2_count', 'mon_num6_mon3_count', 'mon_num6_mon1_mean', 'mon_num6_mon1_max',
#               'mon_num6_mon_2_mean', 'mon_num3_mon1_mean', 'mon_num3_mon1_max', 'mon_num3_mon1_count', 'mon_num12_mon3_max', 'mon_num12_mon3_count',
#               'mon_num12_mon1_mean', 'mon_num12_mon1_max', 'mon_num12_mon1_count', 'mon_num12_mon_2_sum', 'mon_num12_mon_2_min', 'mon_num12_mon_2_mean',
#               'mon_num12_mon_2_count', 'mon_num1_mon3_sum', 'mon_num1_mon3_min', 'mon_num1_mon3_mean', 'mon_num1_mon3_max', 'mon_num1_mon3_count', 'mon_num1_mon1_sum',
#               'mon_num1_mon1_min', 'mon_num1_mon1_mean', 'mon_num1_mon1_max', 'mon_num1_mon1_count', 'mon_num1_mon_2_sum', 'mon_num1_mon_2_min', 'mon_num1_mon_2_mean',
#               'mon_num1_mon_2_max', 'mon_num1_mon_2_count', 'loan_sum', 'loan_hour_diff', 'loan_day_diff', 'limit', 'daydiff6', 'daydiff54', 'daydiff53', 'daydiff52',
#               'daydiff51', 'daydiff5', 'daydiff46', 'daydiff45', 'daydiff44', 'daydiff41', 'daydiff40', 'daydiff4', 'daydiff34', 'daydiff32', 'daydiff30', 'daydiff3',
#               'daydiff20', 'daydiff2', 'daydiff19', 'daydiff13', 'daydiff12', 'daydiff1', 'daydiff0', 'click_pid8_cnt_max', 'click_pid_near_cnt_max',
#               'click_pid_near_cnt_7', 'click_pid_near_cnt_1', 'click_hour_diff', 'click_day_diff', 'cate_price_lv4_max', 'cate_price_lv4_count', 'cate_price_lv3_sum',
#               'cate_price_lv3_min', 'cate_price_lv3_mean', 'cate_price_lv2_sum', 'cate_price_lv2_min', 'cate_price_lv2_count', 'cate_price_lv1_sum',
#               'bins_num6_mon3_count', 'bins_num6_mon1_mean', 'bins_num6_mon1_max', 'bins_num6_mon1_count', 'bins_num6_mon_2_sum', 'bins_num6_mon_2_max',
#               'bins_num3_mon3_mean', 'bins_num3_mon1_mean', 'bins_num3_mon1_max', 'bins_num3_mon1_count', 'bins_num12_mon3_sum', 'bins_num12_mon3_mean',
#               'bins_num12_mon3_max', 'bins_num12_mon3_count', 'bins_num12_mon1_sum', 'bins_num12_mon1_min', 'bins_num12_mon1_max', 'bins_num12_mon1_count',
#               'bins_num12_mon_2_sum', 'bins_num12_mon_2_min', 'bins_num12_mon_2_mean', 'bins_num12_mon_2_max', 'bins_num12_mon_2_count', 'bins_num1_mon3_sum',
#               'bins_num1_mon3_min', 'bins_num1_mon3_mean', 'bins_num1_mon3_max', 'bins_num1_mon1_sum', 'bins_num1_mon1_min', 'bins_num1_mon1_mean',
#               'bins_num1_mon1_max', 'bins_num1_mon1_count', 'bins_num1_mon_2_sum', 'bins_num1_mon_2_min', 'bins_num1_mon_2_count', 'bins_amt_mon3_sum',
#               'bins_amt_mon3_min', 'bins_amt_mon3_mean', 'bins_amt_mon3_max', 'bins_amt_mon3_count', 'bins_amt_mon1_mean', 'bins_amt_mon1_max', 'bins_amt_mon1_count',
#               'bins_amt_mon_2_min', 'bins_amt_mon_2_max', 'bins_amt_mon_2_count', '3_num', '1_num'
              
#             ]

#版本5的变量：综合xgboost跑出的前70变量+版本1变量+版本2变量+剔除与其他变量相关性[-0.3,0.3]之外20以上的变量
# predictors = ['uid','click_hour_diff', 'daydiff6', 'daydiff45', 'daydiff5', 'daydiff53', 'pid_6', 'click_pid_near_cnt_max', 'daydiff3', 'loan_day_diff',
#               'click_pid_near_cnt_1', 'daydiff4', 'mon_price_mon_2_sum', 'cate_price_lv3_sum', 'daydiff20', 'cate_price_lv2_sum', 'daydiff0',
#               'mon_privg_mon_2_mean', 'click_pid_near_cnt_7', 'click_day_diff', 'mon_price_mon3_max', 'daydiff19', 'daydiff1', 'cate_price_lv1_sum',
#               'pid_10', 'cate_price_lv4_count', 'daydiff44', 'daydiff51', 'param_1', 'mon_price_mon3_count', 'bins_num1_mon3_max', 'daydiff2',
#               'cate_price_lv3_mean', 'daydiff13', 'param_16', 'limit', 'cate_price_lv2_count', 'cate_price_lv2_min', 'cate_price_lv3_min',
#               'cate_price_lv4_max', 'bins_num12_mon3_sum', '3_num', 'mon_price_mon3_sum', 'mon_num12_mon3_max', 'daydiff30', 'mon_price_mon_2_min', 'pid_8',
#               'mon_price_mon_2_mean', 'mon_price_mon_2_count', 'mon_price_mon3_min', 'mon_privg_mon_2_max', 'mon_price_mon1_sum', 'bins_amt_mon1_count',
#               'bins_amt_mon3_count', 'bins_num3_mon1_count', 'bins_num6_mon1_count', 'bins_num6_mon_2_sum', 'bins_num12_mon1_min', 'bins_num12_mon_2_sum',
#               'mon_num6_mon1_max', 'mon_num12_mon1_count', 'mon_num12_mon_2_mean', 'click_pid8_cnt_max', 'bins_amt_mon1_max', 'bins_amt_mon3_mean',
#               'bins_num1_mon1_count', 'bins_num1_mon_2_sum', 'bins_num3_mon1_max', 'bins_num6_mon1_max', 'bins_num6_mon3_count', 'bins_num12_mon1_sum',
#               'bins_num12_mon3_count', 'mon_num3_mon1_count', 'mon_num6_mon1_mean', 'mon_num12_mon1_max', 'mon_num12_mon_2_min', 'loan_sum',
#               'bins_amt_mon1_mean', 'bins_amt_mon3_min', 'bins_num1_mon1_max', 'bins_num1_mon3_mean', 'bins_num3_mon1_mean', 'bins_num6_mon1_mean',
#               'bins_num12_mon1_count', 'bins_num12_mon_2_max', 'bins_num12_mon3_max', 'mon_num3_mon1_max', 'mon_num6_mon_2_mean', 'mon_num12_mon1_mean',
#               'mon_num12_mon_2_sum', 'daydiff41', 'bins_amt_mon_2_max', 'bins_num1_mon1_mean', 'bins_num1_mon3_min', 'bins_num3_mon3_mean',
#               'bins_num6_mon_2_max', 'bins_num12_mon1_max', 'bins_num12_mon_2_mean', 'bins_num12_mon3_mean', 'mon_num3_mon1_mean', 'mon_num6_mon3_count',
#               'mon_num12_mon_2_count', 'mon_num12_mon3_count', 'daydiff12', 'aydiff13', 'daydiff32', 'bins_amt_mon_2_min', 'bins_amt_mon3_max',
#               'bins_num1_mon1_min', 'bins_num1_mon1_sum', 'bins_num12_mon_2_count', 'bins_num12_mon_2_min', 'mon_num1_mon_2_max', 'mon_num1_mon_2_sum',
#               'mon_num1_mon3_max'
#             ]


# 版本6的变量：

# predictors = ['uid','loan_sum','loan_hour_diff', 'mons_amt_mon1_mean', 'mons_amt_mon3_min', 'mon_num1_mon3_sum', 'daydiff52', 'mons_amt_mon3_mean', 'mons_amt_mon1_max',
#               'click_hour_diff', 'daydiff6', 'daydiff45', 'daydiff5', 'daydiff53', 'daydiff54', 'pid_6', 'click_pid_near_cnt_max', 'daydiff46',
#               'daydiff3', 'mons_amt_mon3_sum', 'loan_day_diff', 'click_pid_near_cnt_1', 'daydiff4', 'mon_price_mon_2_sum', 'mons_amt_mon3_max',
#               'cate_price_lv3_sum', 'daydiff20', 'cate_price_lv2_sum', 'daydiff0', 'mon_privg_mon_2_mean', 'click_pid_near_cnt_7', 'bins_amt_mon3_sum',
#               'click_day_diff', 'mon_price_mon3_max', 'daydiff19', 'daydiff1', 'cate_price_lv1_sum', 'pid_10', 'mon_num1_mon_2_count',
#               'cate_price_lv4_count', 'mon_num1_mon1_mean', 'daydiff44', 'daydiff51', 'param_1', 'mon_price_mon3_count', 'bins_num1_mon3_max',
#               'daydiff2', 'cate_price_lv3_mean', 'daydiff13', 'param_16', 'limit', 'cate_price_lv2_count', 'cate_price_lv2_min', 'cate_price_lv3_min',
#               'cate_price_lv4_max', 'bins_num12_mon3_sum', 'mons_amt_mon_2_min', '3_num', 'bins_num1_mon_2_min', 'mon_price_mon3_sum',
#               'bins_num1_mon3_sum', 'mon_num12_mon3_max', 'mon_num1_mon3_min', 'daydiff30', 'mon_price_mon_2_min', 'pid_8', 'mon_num1_mon_2_min',
#               'mon_price_mon_2_mean', 'mon_price_mon_2_count', 'mon_price_mon3_min', 'mon_privg_mon_2_max', 'mon_price_mon1_sum'
  
# ]


sample_demand = sample_demand.loc[:,predictors]

In [6]:
sample_demand.columns

Index(['uid', 'loan_hour_diff', 'loan_day_diff', 'bins_amt_mon1_count',
       'bins_amt_mon1_max', 'bins_amt_mon1_mean', 'bins_amt_mon_2_count',
       'bins_amt_mon_2_min', 'bins_amt_mon3_max', 'bins_amt_mon3_mean',
       'bins_amt_mon3_min', 'mons_amt_mon1_count', 'mons_amt_mon1_max',
       'mons_amt_mon1_mean', 'mons_amt_mon3_mean', 'mons_amt_mon3_min',
       'mons_amt_mon3_sum', 'bins_num1_mon1_count', 'bins_num1_mon1_max',
       'bins_num1_mon1_mean', 'bins_num1_mon1_min', 'bins_num1_mon1_sum',
       'bins_num1_mon_2_count', 'bins_num1_mon3_mean', 'bins_num1_mon3_min',
       'bins_num1_mon3_sum', 'bins_num12_mon_2_count', 'bins_num12_mon_2_min',
       'bins_num12_mon3_max', 'bins_num12_mon3_sum', 'mon_num1_mon1_count',
       'mon_num1_mon1_max', 'mon_num1_mon1_mean', 'mon_num1_mon1_min',
       'mon_num1_mon1_sum', 'mon_num1_mon_2_count', 'mon_num1_mon_2_max',
       'mon_num1_mon_2_mean', 'mon_num1_mon_2_min', 'mon_num1_mon_2_sum',
       'mon_num1_mon3_count', 'mon_num1

In [7]:
len(list(sample_demand.columns))

72

In [87]:
len(sample_demand)

77060

In [7]:
predictors = [pre for pre in predictors if pre not in ['is_demand','loan_sum']]
pred_demand = pred_demand.loc[:,predictors]

In [9]:
len(list(pred_demand.columns))

71

In [8]:
########################
#版本一：先需求再需求量，效果没直接回归好
# xgb1 = train_cla_model(sample_demand)
# is_demand = read_pred_demand_data(xgb1,pred_demand)
# sample_demand = sample_demand.drop(['is_demand'],axis=1)
# sample_loan_reg = pd.merge(sample_demand,is_demand,on=['uid'],how='left')
# sample_loan_reg = sample_loan_reg[sample_loan_reg['uid'].isin(t_loan.uid)] #缩小样本范围
# pred_loan_new = pd.merge(pred_demand,is_demand,on='uid',how='left')
# sample_loan_reg = sample_loan_reg.fillna(0)
# xgb2,compare,fea_imp  = train_reg_model(sample_loan_reg)
########################

xgb2,compare,fea_imp = train_reg_model(sample_demand)

#1. AUC ：0.850654557699， reg:2.55509742274  线上：2.055245
#2. 删除共线性为1的特征，筛选出与loan_sum相关性大于|cor|>0.1的特征，AUC:0.842948012399,reg:2.57904679754,线上：2.052592
#3. 不分类直接回归,reg: 1.9009753967529406  线上：1.809802
#4.第三步的基础上不做变量的处理，reg: 1.8995983786790303 线上：1.814778
#5.对第四部的一些相关系数为1的变量进行剔除，reg:1.8991801181062316  线上: 1.820631
#########################
#取上述步骤3的结果调参1.8995983786790303 
#1.reg树棵树1000，reg: 3.62256034832 线上：
#--------------changhong-------------------------------------------
#2.reg树棵树1000，版本2的变量，reg: 1.9022556420917773 线上：1.812433
#!!!!3.reg树棵树500，版本2的变量，reg:  线上：1.9005198006834867 线上：1.80916
#4.第三步变量的基础上，通过调整参数，进一步训练模型reg: 1.9013367205053058 线上： 
#5.第四步的基础上，通过对预测值取绝对值，而不是小于0的取值为0，reg:1.9011295760313531 线上：
#6.第5步的基础上，subsample=0.6,colsample_bytree=0.6，reg:1.9004985324025576 线上：1.809126
#7.第5步的基础上,特征变成xgboost最优70个特征，reg：1.8998386615685263，线上：1.809832  （线上表现不太好 可能是因为XGBoost调参是根据版本2的调的）
#8.选取之前的滑窗数据直接回归，reg: 1.4361484634841084,加上is_demand 的reg：1.4336125069581849，线上：

#------------------DR------------------------------------------------
#3.reg树棵树500，版本4的变量，reg:3.60823217642 线上：1.80991
#4.reg树棵树500，版本5的变量，reg:1.9072625994563375 线上：1.816843

the result of regression is: 1.9004985324025576


In [9]:
result = get_result(xgb2,pred_demand)
result.to_csv(file_dir+'Loan_Forecasting_Upload_Sample.csv',header=False,index=False)

tune 参数

In [60]:
X = sample_demand.loc[:,predictors]
y = sample_demand.loc[:,'loan_sum']
# X = X[predictors].fillna(0)
xgb_liner = xgb.XGBRegressor(learning_rate =0.01,n_estimators=500,max_depth=5,min_child_weight=7,gamma=0.1,subsample=0.5,
                             colsample_bytree=0.5,reg_alpha=1,objective= 'reg:linear',nthread=4,scale_pos_weight=1,seed=1)

In [49]:
def step_1(xgb_liner):
   
    start_time = time.time()
#     xgb_liner.set_params(**prior_params)
    param_test1 = {
                    'max_depth':list(range(3,10,2)),
                    'min_child_weight':list(range(1,8,2))
                   }
    gsearch1 = GridSearchCV(xgb_liner,param_grid = param_test1, scoring='mean_squared_error',iid=False, cv=5)
    gsearch1.fit(X,y)
    print("\nStep1: The process of tuning max_depth & min_child_weigh:")
    print(gsearch1.grid_scores_)
    print(gsearch1.best_params_)
    print(gsearch1.best_score_)
    params = gsearch1.best_params_
#    prior_params_new = dict(prior_params,**params)
    end_time = time.time()
    print("\nThe time step1 used is:",end_time - start_time)

    return params

In [50]:
prior_params1 = step_1(xgb_liner)

D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-pa

D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-pa

D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-pa

D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-pa

D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-pa


Step1: The process of tuning max_depth & min_child_weigh:
[mean: -3.63313, std: 0.04979, params: {'max_depth': 4, 'min_child_weight': 1}, mean: -3.63091, std: 0.04787, params: {'max_depth': 4, 'min_child_weight': 3}, mean: -3.62990, std: 0.04723, params: {'max_depth': 4, 'min_child_weight': 5}, mean: -3.62925, std: 0.04773, params: {'max_depth': 4, 'min_child_weight': 7}, mean: -3.63059, std: 0.05299, params: {'max_depth': 5, 'min_child_weight': 1}, mean: -3.63069, std: 0.05208, params: {'max_depth': 5, 'min_child_weight': 3}, mean: -3.62718, std: 0.04901, params: {'max_depth': 5, 'min_child_weight': 5}, mean: -3.62477, std: 0.04596, params: {'max_depth': 5, 'min_child_weight': 7}, mean: -3.63004, std: 0.05116, params: {'max_depth': 6, 'min_child_weight': 1}, mean: -3.63552, std: 0.05627, params: {'max_depth': 6, 'min_child_weight': 3}, mean: -3.63024, std: 0.05016, params: {'max_depth': 6, 'min_child_weight': 5}, mean: -3.62579, std: 0.04642, params: {'max_depth': 6, 'min_child_weigh

In [51]:
prior_params1 

{'max_depth': 5, 'min_child_weight': 7}

In [54]:
def step_2(prior_params):

    start_time = time.time()
    xgb_liner.set_params(**prior_params)
    param_test2 = {
                   'gamma': [0,0.1,0.2,0.3,0.4,0.5]
                   }
    gsearch2 = GridSearchCV(xgb_liner,param_grid = param_test2, scoring='mean_squared_error',iid=False, cv=5)
    gsearch2.fit(X,y)
    print("\nStep2:The process of tuning gamma:")
    print(gsearch2.grid_scores_)
    print(gsearch2.best_params_)
    print(gsearch2.best_score_)
    params = gsearch2.best_params_
    prior_params_new = dict(prior_params,**params)
    end_time = time.time()
    print("\nThe time step2 used is:",end_time - start_time)
   
    return prior_params_new

In [55]:
prior_params2 = step_2(prior_params1 )

D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-pa


Step2:The process of tuning gamma:
[mean: -3.62477, std: 0.04596, params: {'gamma': 0}, mean: -3.62450, std: 0.04611, params: {'gamma': 0.1}, mean: -3.62462, std: 0.04615, params: {'gamma': 0.2}, mean: -3.62465, std: 0.04616, params: {'gamma': 0.3}, mean: -3.62467, std: 0.04620, params: {'gamma': 0.4}, mean: -3.62461, std: 0.04604, params: {'gamma': 0.5}]
{'gamma': 0.1}
-3.6245033780304405

The time step2 used is: 562.5715589523315


In [63]:
def step_3(prior_params):

    start_time = time.time()
    xgb_liner.set_params(**prior_params)
    param_test3 = {
                    'subsample':[i/10.0 for i in range(4,6)],
                    'colsample_bytree':[i/10.0 for i in range(4,6)]
                   }

    gsearch3 = GridSearchCV(xgb_liner,param_grid = param_test3, scoring='mean_squared_error',iid=False, cv=5)
    gsearch3.fit(X,y)
    print("\nStep3:The process of tuning subsample&colsample_bytree:")
    print(gsearch3.grid_scores_)
    print(gsearch3.best_params_)
    print(gsearch3.best_score_)
    params = gsearch3.best_params_
    prior_params_new = dict(prior_params,**params)
    end_time = time.time()
    print("\nThe time step3 used is:",end_time - start_time)
   
    return prior_params_new

In [64]:
prior_params3 = step_3(prior_params2)

D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-pa


Step3:The process of tuning subsample&colsample_bytree:
[mean: -3.63169, std: 0.04922, params: {'colsample_bytree': 0.4, 'subsample': 0.4}, mean: -3.62659, std: 0.04663, params: {'colsample_bytree': 0.4, 'subsample': 0.5}, mean: -3.62867, std: 0.04591, params: {'colsample_bytree': 0.5, 'subsample': 0.4}, mean: -3.62621, std: 0.04695, params: {'colsample_bytree': 0.5, 'subsample': 0.5}]
{'colsample_bytree': 0.5, 'subsample': 0.5}
-3.626205478881174

The time step3 used is: 326.02959990501404


In [71]:
def step_4(prior_params):
   
    start_time = time.time()
    xgb_liner.set_params(**prior_params)
    param_test4 = {
                    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
                   }
    gsearch4 = GridSearchCV(xgb_liner,param_grid = param_test4, scoring='mean_squared_error',iid=False, cv=5)
    gsearch4.fit(X,y)
    print("\nStep4:The process of tuning reg_alpha:")
    print(gsearch4.grid_scores_)
    print(gsearch4.best_params_)
    print(gsearch4.best_score_)
    params = gsearch4.best_params_
    prior_params_new = dict(prior_params,**params)
#     joblib.dump(gsearch4.best_estimator_, '{}{}_XGB_{}.model'.format(file_dir,lend_id,today))
    end_time = time.time()
    print("\nThe time step4 used is:",end_time - start_time)
   
    return prior_params_new,gsearch4.best_score_

In [72]:
prior_params4 = step_4(prior_params3)

D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-packages\sklearn\metrics\scorer.py:90: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
D:\programFiles\anaconda\lib\site-pa


Step4:The process of tuning reg_alpha:
[mean: -3.62681, std: 0.04756, params: {'reg_alpha': 1e-05}, mean: -3.62667, std: 0.04692, params: {'reg_alpha': 0.01}, mean: -3.62643, std: 0.04699, params: {'reg_alpha': 0.1}, mean: -3.62621, std: 0.04695, params: {'reg_alpha': 1}, mean: -3.63429, std: 0.04813, params: {'reg_alpha': 100}]
{'reg_alpha': 1}
-3.626205478881174

The time step4 used is: 387.2895655632019


In [ ]:
# fea_imp.sort_values(by=1,ascending=False).to_csv('fea_imp.csv')
# sample_loan_reg.corr().to_csv('demand_corr.csv')

#### 数据预处理

In [ ]:
#训练集
t_user = pd.read_csv(file_dir+'t_user.csv')  #用户信息表，用户量90993
t_loan_sum = pd.read_csv(file_dir+'t_loan_sum.csv')
t_order_10 = pd.read_hdf("{}/data_v2/details_data/t_order_10.h5".format(file_dir), key = "t_order_10")
t_click_10 = pd.read_hdf("{}/data_v2/details_data/t_click_agg_10.h5".format(file_dir), key = "t_click_agg_10")
t_loan_10 = pd.read_hdf("{}/data_v2/details_data/t_loan_10.h5".format(file_dir), key = "t_loan_10")
#测试集
t_order_11 = pd.read_hdf("{}/data_v2/details_data/t_order_11.h5".format(file_dir), key = "t_order_11")
t_click_11 = pd.read_hdf("{}/data_v2/details_data/t_click_agg_11.h5".format(file_dir), key = "t_click_agg_11")
t_loan_11 = pd.read_hdf("{}/data_v2/details_data/t_loan_11.h5".format(file_dir), key = "t_loan_11")

#生成训练数据
#sample_demand
sample_demand = list(set(t_click_10.uid))
sample_demand = pd.DataFrame(sample_demand).rename(columns={0:'uid'})
sample_demand['is_demand'] = 0
sample_demand['is_demand'] = sample_demand[sample_demand['uid'].isin(set(t_loan_sum.uid))]['is_demand'].replace(0,1)
sample_demand['is_demand'] = sample_demand['is_demand'].fillna(0)
sample_demand.is_demand = sample_demand.is_demand.astype('int32')
sample_demand = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(sample_demand,t_user,on='uid',how='left'),t_order_10,on='uid',how='left'),
                                           t_click_10,on='uid',how='left'),t_loan_10,on='uid',how='left'),t_loan_sum.loc[:,['uid','loan_sum']],on='uid',how='left')
sample_demand['loan_sum'] = sample_demand['loan_sum'].fillna(0)
# sample_demand['active_buy_diff'] = (pd.to_datetime(sample_demand.loc[:,'buy_time_min'])- pd.to_datetime(sample_demand.loc[:,'active_date']))/np.timedelta64(1,'D')
sample_demand.to_hdf("{}/sample_demand_v2_2.h5".format(file_dir), key = "sample_demand_v2_2", complib = "zlib", complevel = 1, mode = "w")

#生成测试数据
pred_demand = pd.merge(pd.merge(pd.merge(t_user,t_order_11,on='uid',how='left'),t_click_11,on='uid',how='left'),t_loan_11,on='uid',how='left')
pred_demand['active_buy_diff'] = (pd.to_datetime(pred_demand.loc[:,'buy_time_min'])- pd.to_datetime(pred_demand.loc[:,'active_date']))/np.timedelta64(1,'D')
pred_demand.to_hdf("{}/pred_demand_v2_2.h5".format(file_dir), key = "pred_demand_v2_2", complib = "zlib", complevel = 1, mode = "w")


In [167]:
def get_day_diffs(data):
    
    day_cols = ['active_date','buy_time_max','buy_time_min', 'max_buy_time','click_day_max', 'click_day_min','loan_day_max', 'loan_day_min']
    new_cols = ['daydiff' + str(i) for i in range(0,len(day_cols)*(len(day_cols)-1))]
    s = -1
    for i in range(0,len(day_cols)):
        for j in range(1,len(day_cols)):
            s = s+1
            day1,day2 = day_cols[i],day_cols[j]
            data[new_cols[s]] = abs(pd.to_datetime(data[day1])- pd.to_datetime(data[day2]))/np.timedelta64(1,'D')
            

    data = data.drop(day_cols,axis=1)
    
    return data 

In [132]:
# sample_demand.columns
date_columns = list(pd.DataFrame(sample_demand.dtypes)[pd.DataFrame(sample_demand.dtypes)[0]=='object'].index)

In [172]:
sample_demand = get_day_diffs(sample_demand)
sample_demand = sample_demand.drop(['buy_diff_bins', 'loan_diff_bins'],axis=1)
pred_demand = get_day_diffs(pred_demand)
pred_demand = pred_demand.drop(['buy_diff_bins', 'loan_diff_bins'],axis=1)

In [187]:
sample_demand.to_hdf("{}/sample_demand_v2_3.h5".format(file_dir), key = "sample_demand_v2_3", complib = "zlib", complevel = 1, mode = "w")
pred_demand.to_hdf("{}/pred_demand_v2_3.h5".format(file_dir), key = "pred_demand_v2_3", complib = "zlib", complevel = 1, mode = "w")

D:\programFiles\anaconda\lib\site-packages\pandas\core\generic.py:1282: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->interval,key->block2_values] [items->['click_diff_bins']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [183]:
preloan_cols = ['uid','is_demand','click_hour_diff','click_pid8_cnt_max','click_day_diff','loan_hour_diff',
                'loan_day_diff','bins_amt_mon1_count','bins_amt_mon1_max','bins_amt_mon1_mean','bins_amt_mon_2_max',
                'bins_amt_mon_2_sum','bins_amt_mon3_count','bins_amt_mon3_mean','bins_amt_mon3_min','bins_amt_mon3_sum',
                'mons_amt_mon1_count','mons_amt_mon1_max','mons_amt_mon1_mean','mons_amt_mon3_max','mons_amt_mon3_mean',
                'mons_amt_mon3_min','mons_amt_mon3_sum','bins_num1_mon1_count','bins_num1_mon1_max','bins_num1_mon1_mean',
                'bins_num1_mon1_sum','bins_num1_mon_2_count','bins_num1_mon_2_sum','bins_num1_mon3_mean','bins_num1_mon3_min',
                'bins_num1_mon3_sum','bins_num3_mon1_count','bins_num3_mon1_max','bins_num3_mon1_mean','bins_num3_mon3_mean',
                'bins_num3_mon3_sum','bins_num6_mon1_count','bins_num6_mon1_max','bins_num6_mon1_mean','bins_num6_mon_2_max',
                'bins_num6_mon_2_mean','bins_num6_mon_2_sum','bins_num6_mon3_count','bins_num12_mon1_count','bins_num12_mon1_max',
                'bins_num12_mon1_mean','bins_num12_mon1_min','bins_num12_mon1_sum','bins_num12_mon_2_max','bins_num12_mon_2_mean',
                'bins_num12_mon_2_min','bins_num12_mon_2_sum','bins_num12_mon3_count','bins_num12_mon3_max','bins_num12_mon3_mean',
                'bins_num12_mon3_min','bins_num12_mon3_sum','mon_num1_mon1_count','mon_num1_mon1_max','mon_num1_mon1_mean',
                'mon_num1_mon1_min','mon_num1_mon1_sum','mon_num1_mon_2_count','mon_num1_mon_2_min','mon_num1_mon3_mean',
                'mon_num1_mon3_min','mon_num1_mon3_sum','mon_num3_mon1_count','mon_num3_mon1_max','mon_num3_mon1_mean',
                'mon_num6_mon1_count','mon_num6_mon1_max','mon_num6_mon1_mean','mon_num6_mon_2_mean','mon_num6_mon3_count',
                'mon_num6_mon3_max','mon_num12_mon1_count','mon_num12_mon1_max','mon_num12_mon1_mean','mon_num12_mon_2_count',
                'mon_num12_mon_2_max','mon_num12_mon_2_mean','mon_num12_mon_2_min','mon_num12_mon_2_sum','mon_num12_mon3_count',
                'mon_num12_mon3_max','1_num','loan_sum','daydiff5','daydiff12','daydiff13','daydiff19','daydiff20','daydiff32',
                'daydiff33','daydiff34','daydiff40','daydiff40','daydiff41'
            ]

In [184]:
sample_demand_0 = sample_demand.loc[:,strong_cols]
sample_demand_0 = sample_demand_0.corr()
sample_demand_0.to_csv('cor_v1.csv')